In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [90]:
data = pd.read_csv('dataset.csv', sep=',')

In [37]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

header = ["color", "diameter", "label"]

In [30]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

def class_counts(rows):
    counts={}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [43]:
class Question:

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [47]:
q = Question(0, 'Green')
print(q)
example = training_data[0]
print(example)
q.match(example)

Is color == Green?
['Green', 3, 'Apple']


True

In [48]:
def partition(rows, question):
    true_rows, false_rows =[], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [49]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))

In [50]:
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [51]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [54]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] /float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [58]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)

In [61]:
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Green'))

info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [62]:
def find_best_split(rows):
    best_gain = 0  
    best_question = None 
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  
        values = set([row[col] for row in rows])
        for val in values:  
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)          
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

In [63]:
best_gain, best_question = find_best_split(training_data)
best_question

Is diameter >= 3?

In [85]:

class Leaf:
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
class Decision_Node:
    

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
def build_tree(rows):
    gain, question = find_best_split(rows)

    
    if gain == 0:
        return Leaf(rows)

    
    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [86]:
def build_tree(rows):
    gain, question = find_best_split(rows)

   
    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

  
    true_branch = build_tree(true_rows)

  
    false_branch = build_tree(false_rows)

   
    return Decision_Node(question, true_branch, false_branch)

In [89]:
my_tree = build_tree(data)

KeyError: 0

In [88]:
print_tree(my_tree)

Is diameter >= 3?
---> True:
   Is color == Yellow?
   ---> True:
      Predict {'Apple': 1, 'Lemon': 1}
   ---> Flase:
      Predict {'Apple': 1, 'Lemon': 1}
---> Flase:
   Is color == Yellow?
   ---> True:
      Predict {'Apple': 1, 'Lemon': 1}
   ---> Flase:
      Predict {'Apple': 1, 'Lemon': 1}
